# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


browser = webdriver.Chrome()

# entrar no google e pesquisar cotacao do dolar
browser.get("https://www.google.com/")
browser.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao dolar" ,Keys.ENTER )

# pegar cotacao do dolar
dolar = browser.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(dolar)

# pesquisar cotacao euro
browser.get("https://www.google.com/")
browser.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotacao euro",Keys.ENTER)

# pegar cotacoa do euro
euro = browser.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(euro)

# pesquisar cotacao do ouro (melhorcambio)
browser.get("https://www.melhorcambio.com/ouro-hoje")

# pegar cotacao do ouro
ouro = browser.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value')
ouro = ouro.replace(",",".")
print(ouro)


browser.quit()


5.0115
5.50793919
324.63


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [9]:
# importar base de dados
import pandas as pd

table = pd.read_excel("Produtos.xlsx")
#display(table)

- Atualizando os preços e o cálculo do Preço Final

In [10]:
# atalizar cotacoes
table.loc[table["Moeda"] == "Dólar","Cotação"] = float(dolar)
table.loc[table["Moeda"] == "Euro","Cotação"] = float(euro)
table.loc[table["Moeda"] == "Ouro","Cotação"] = float(ouro)

table["Preço de Compra"] = table["Preço Original"] * table ["Cotação"]

table["Preço de Venda"] = table["Preço de Compra"] * table ["Margem"]
display(table)

# atualizar preco de compra e venda

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.011500,5011.449885,1.40,7016.029839
1,Carro Renault,4500.00,Euro,5.507939,24785.726355,2.00,49571.452710
2,Notebook Dell,899.99,Dólar,5.011500,4510.299885,1.70,7667.509805
3,IPhone,799.00,Dólar,5.011500,4004.188500,1.70,6807.120450
4,Carro Fiat,3000.00,Euro,5.507939,16523.817570,1.90,31395.253383
5,Celular Xiaomi,480.48,Dólar,5.011500,2407.925520,2.00,4815.851040
6,Joia 20g,20.00,Ouro,324.630000,6492.600000,1.15,7466.490000


### Agora vamos exportar a nova base de preços atualizada

In [11]:
# exportar base com precos atualizados
table.to_excel("ProdutosNovo.xlsx", index = False)